# Dealing with imbalanced datasets, combining oversampling with VAE and undersampling to improve model recognition over all classes.  

Import packages, classifiers and etc.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.metrics import confusion_matrix, make_scorer
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from collections import Counter

from imblearn.over_sampling import SMOTE, RandomOverSampler

Import VAEOversampler.

In [2]:
from VAEOversampler import VAEOversampler

2023-07-10 22:29:41.990707: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-10 22:29:41.990727: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data  
You can load some dataset from Imbalanced Learn list (https://imbalanced-learn.org/stable/datasets/index.html) or use your own data.  


In [3]:
from imblearn.datasets import fetch_datasets

dset_name = 'thyroid_sick'
dset = fetch_datasets()[dset_name]

X, y = StandardScaler().fit_transform(dset.data), dset.target

In [4]:
X

array([[-0.55883917,  0.72178095, -0.65895412, ...,  2.96175973,
        -0.10221233, -0.61453065],
       [-1.50655984,  0.72178095, -0.65895412, ..., -0.33763711,
        -0.10221233, -0.61453065],
       [-0.29558343, -1.38546189,  1.51755634, ..., -0.33763711,
        -0.10221233, -0.61453065],
       ...,
       [ 1.17864871,  0.72178095, -0.65895412, ..., -0.33763711,
        -0.10221233, -0.61453065],
       [ 1.07334641, -1.38546189,  1.51755634, ..., -0.33763711,
        -0.10221233,  1.62725814],
       [ 0.65213723,  0.72178095, -0.65895412, ..., -0.33763711,
        -0.10221233, -0.61453065]])

In [5]:
y[y == -1] = 0

We split data into train and test partitions.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

This is a simple function to undersample freely.  

In [7]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res

    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)

    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [8]:
def train_val(X, y, Xt, yt, random_state=42):
    classifiers = {
        "CatBoostClassifier": CatBoostClassifier(verbose=False, random_seed=random_state),
        "LGBMClassifier": LGBMClassifier(random_state=random_state),
        "XGBClassifier": XGBClassifier(random_state=random_state),
        "BaggingClassifier": BaggingClassifier(random_state=random_state),
        "RandomForestClassifier": RandomForestClassifier(random_state=random_state),
    }
    scores = []
    predictions = []
    for key, classifier in classifiers.items():
        print('_' * 50)
        name = key
        classifier.fit(X, y)
        print("Classifier: ", name)
        y_pred = classifier.predict(Xt)
        cm = confusion_matrix(yt, y_pred)
        print(cm)
        print('')
        predictions.append(y_pred)
        tn = cm[0,0]
        fp = cm[0,1]
        fn = cm[1,0]
        tp = cm[1,1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)
        scores.append(tnr * tpr)
        print('TNR:', round(tnr, 5))
        print('TPR:', round(tpr, 5))
        print('TNRxTPR:', round(tnr * tpr, 5))
        print('G-mean:', round(np.sqrt(tnr * tpr), 5))

    print('_' * 50)
    print('Ensemble predictions (majority voting):')
    predictions = np.sum(predictions, axis=0)
    predictions[predictions < 3] = 0
    predictions[predictions >= 3] = 1

    cm = confusion_matrix(yt, predictions)
    print(cm)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)
    print('')
    print('TNR:', round(tnr, 5))
    print('TPR:', round(tpr, 5))
    print('TNRxTPR:', round(tnr * tpr, 5))
    print('G-mean:', round(np.sqrt(tnr * tpr), 5))


## Without resampling (base line) 
Which is starting score?


In [9]:
print('Original dataset shape %s' % Counter(y))
print('Ratio->', round(Counter(y)[0]/Counter(y)[1], 1), ': 1')

Original dataset shape Counter({0: 3541, 1: 231})
Ratio-> 15.3 : 1


In [10]:
train_val(X_train, y_train, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[708   1]
 [  3  43]]

TNR: 0.99859
TPR: 0.93478
TNRxTPR: 0.93346
G-mean: 0.96616
__________________________________________________
Classifier:  LGBMClassifier
[[705   4]
 [  5  41]]

TNR: 0.99436
TPR: 0.8913
TNRxTPR: 0.88628
G-mean: 0.94142
__________________________________________________
Classifier:  XGBClassifier
[[706   3]
 [  4  42]]

TNR: 0.99577
TPR: 0.91304
TNRxTPR: 0.90918
G-mean: 0.95351
__________________________________________________
Classifier:  BaggingClassifier
[[705   4]
 [  5  41]]

TNR: 0.99436
TPR: 0.8913
TNRxTPR: 0.88628
G-mean: 0.94142
__________________________________________________
Classifier:  RandomForestClassifier
[[708   1]
 [  9  37]]

TNR: 0.99859
TPR: 0.80435
TNRxTPR: 0.80321
G-mean: 0.89622
__________________________________________________
Ensemble predictions (majority voting):
[[708   1]
 [  4  42]]

TNR: 0.99859
TPR: 0.91304
TNRxTPR: 0.91176
G-mean: 0.95486


## Ratio 1:1  
Let's see classifiers scores when dataset is balanced.  


In [11]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fe134351ca0>
Traceback (most recent call last):
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [12]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 2832, 1: 2832})
Ratio->  1 : 1.0


In [13]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[702   7]
 [  0  46]]

TNR: 0.99013
TPR: 1.0
TNRxTPR: 0.99013
G-mean: 0.99505
__________________________________________________
Classifier:  LGBMClassifier
[[704   5]
 [  0  46]]

TNR: 0.99295
TPR: 1.0
TNRxTPR: 0.99295
G-mean: 0.99647
__________________________________________________
Classifier:  XGBClassifier
[[703   6]
 [  0  46]]

TNR: 0.99154
TPR: 1.0
TNRxTPR: 0.99154
G-mean: 0.99576
__________________________________________________
Classifier:  BaggingClassifier
[[705   4]
 [  4  42]]

TNR: 0.99436
TPR: 0.91304
TNRxTPR: 0.90789
G-mean: 0.95283
__________________________________________________
Classifier:  RandomForestClassifier
[[705   4]
 [  2  44]]

TNR: 0.99436
TPR: 0.95652
TNRxTPR: 0.95113
G-mean: 0.97526
__________________________________________________
Ensemble predictions (majority voting):
[[704   5]
 [  0  46]]

TNR: 0.99295
TPR: 1.0
TNRxTPR: 0.99295
G-mean: 0.99647


In [14]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [15]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 2832, 1: 2832})
Ratio->  1 : 1.0


In [16]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[704   5]
 [  2  44]]

TNR: 0.99295
TPR: 0.95652
TNRxTPR: 0.94978
G-mean: 0.97456
__________________________________________________
Classifier:  LGBMClassifier
[[707   2]
 [  3  43]]

TNR: 0.99718
TPR: 0.93478
TNRxTPR: 0.93215
G-mean: 0.96548
__________________________________________________
Classifier:  XGBClassifier
[[706   3]
 [  2  44]]

TNR: 0.99577
TPR: 0.95652
TNRxTPR: 0.95247
G-mean: 0.97595
__________________________________________________
Classifier:  BaggingClassifier
[[707   2]
 [  4  42]]

TNR: 0.99718
TPR: 0.91304
TNRxTPR: 0.91047
G-mean: 0.95418
__________________________________________________
Classifier:  RandomForestClassifier
[[707   2]
 [  3  43]]

TNR: 0.99718
TPR: 0.93478
TNRxTPR: 0.93215
G-mean: 0.96548
__________________________________________________
Ensemble predictions (majority voting):
[[707   2]
 [  3  43]]

TNR: 0.99718
TPR: 0.93478
TNRxTPR: 0.93215
G-mean: 0.96548


In [17]:
# VAEOversampler

vae_sampler = VAEOversampler(epochs=500,
                              intermediate_dim=512,
                              batch_size=64,
                              random_state=42,
                              verbose=False)
X_res, y_res = vae_sampler.fit_resample(X_train, y_train)

2023-07-10 22:30:43.772008: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-10 22:30:43.772035: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-10 22:30:43.772059: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a-Modern-15-A5M): /proc/driver/nvidia/version does not exist
2023-07-10 22:30:43.772284: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0.0: 2832, 1.0: 2832})
Ratio->  1 : 1.0


In [19]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[707   2]
 [  2  44]]

TNR: 0.99718
TPR: 0.95652
TNRxTPR: 0.95382
G-mean: 0.97664
__________________________________________________
Classifier:  LGBMClassifier
[[707   2]
 [  3  43]]

TNR: 0.99718
TPR: 0.93478
TNRxTPR: 0.93215
G-mean: 0.96548
__________________________________________________
Classifier:  XGBClassifier
[[707   2]
 [  5  41]]

TNR: 0.99718
TPR: 0.8913
TNRxTPR: 0.88879
G-mean: 0.94276
__________________________________________________
Classifier:  BaggingClassifier
[[707   2]
 [  7  39]]

TNR: 0.99718
TPR: 0.84783
TNRxTPR: 0.84543
G-mean: 0.91948
__________________________________________________
Classifier:  RandomForestClassifier
[[708   1]
 [  9  37]]

TNR: 0.99859
TPR: 0.80435
TNRxTPR: 0.80321
G-mean: 0.89622
__________________________________________________
Ensemble predictions (majority voting):
[[708   1]
 [  4  42]]

TNR: 0.99859
TPR: 0.91304
TNRxTPR: 0.91176
G-mean: 0.95486


## Under/Oversampling combination  
Now we can tuning the number of instances for each class to optimize metric.  


In [20]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fe13d414af0>
Traceback (most recent call last):
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [21]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.9)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 2832, 0: 2549})
Ratio->  1 : 1.1


In [22]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[702   7]
 [  0  46]]

TNR: 0.99013
TPR: 1.0
TNRxTPR: 0.99013
G-mean: 0.99505
__________________________________________________
Classifier:  LGBMClassifier
[[703   6]
 [  0  46]]

TNR: 0.99154
TPR: 1.0
TNRxTPR: 0.99154
G-mean: 0.99576
__________________________________________________
Classifier:  XGBClassifier
[[704   5]
 [  0  46]]

TNR: 0.99295
TPR: 1.0
TNRxTPR: 0.99295
G-mean: 0.99647
__________________________________________________
Classifier:  BaggingClassifier
[[704   5]
 [  3  43]]

TNR: 0.99295
TPR: 0.93478
TNRxTPR: 0.92819
G-mean: 0.96343
__________________________________________________
Classifier:  RandomForestClassifier
[[704   5]
 [  2  44]]

TNR: 0.99295
TPR: 0.95652
TNRxTPR: 0.94978
G-mean: 0.97456
__________________________________________________
Ensemble predictions (majority voting):
[[704   5]
 [  0  46]]

TNR: 0.99295
TPR: 1.0
TNRxTPR: 0.99295
G-mean: 0.99647


In [26]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [27]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.6)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 2832, 0: 1699})
Ratio->  1 : 1.7


In [28]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[701   8]
 [  0  46]]

TNR: 0.98872
TPR: 1.0
TNRxTPR: 0.98872
G-mean: 0.99434
__________________________________________________
Classifier:  LGBMClassifier
[[703   6]
 [  2  44]]

TNR: 0.99154
TPR: 0.95652
TNRxTPR: 0.94843
G-mean: 0.97387
__________________________________________________
Classifier:  XGBClassifier
[[702   7]
 [  1  45]]

TNR: 0.99013
TPR: 0.97826
TNRxTPR: 0.9686
G-mean: 0.98418
__________________________________________________
Classifier:  BaggingClassifier
[[702   7]
 [  1  45]]

TNR: 0.99013
TPR: 0.97826
TNRxTPR: 0.9686
G-mean: 0.98418
__________________________________________________
Classifier:  RandomForestClassifier
[[706   3]
 [  3  43]]

TNR: 0.99577
TPR: 0.93478
TNRxTPR: 0.93083
G-mean: 0.96479
__________________________________________________
Ensemble predictions (majority voting):
[[704   5]
 [  1  45]]

TNR: 0.99295
TPR: 0.97826
TNRxTPR: 0.97136
G-mean: 0.98558


In [47]:
# VAEOversampler

X_res, y_res = vae_sampler.resample(X_train, y_train, sampling_strategy=3)

In [48]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.15)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1.0: 8126, 0.0: 425})
Ratio->  1 : 19.1


In [49]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[698  11]
 [  0  46]]

TNR: 0.98449
TPR: 1.0
TNRxTPR: 0.98449
G-mean: 0.99221
__________________________________________________
Classifier:  LGBMClassifier
[[692  17]
 [  2  44]]

TNR: 0.97602
TPR: 0.95652
TNRxTPR: 0.93359
G-mean: 0.96622
__________________________________________________
Classifier:  XGBClassifier
[[695  14]
 [  1  45]]

TNR: 0.98025
TPR: 0.97826
TNRxTPR: 0.95894
G-mean: 0.97926
__________________________________________________
Classifier:  BaggingClassifier
[[699  10]
 [  1  45]]

TNR: 0.9859
TPR: 0.97826
TNRxTPR: 0.96446
G-mean: 0.98207
__________________________________________________
Classifier:  RandomForestClassifier
[[697  12]
 [  1  45]]

TNR: 0.98307
TPR: 0.97826
TNRxTPR: 0.9617
G-mean: 0.98066
__________________________________________________
Ensemble predictions (majority voting):
[[700   9]
 [  0  46]]

TNR: 0.98731
TPR: 1.0
TNRxTPR: 0.98731
G-mean: 0.99363


## References  

  - Classification with Imbalanced Datasets:  
    https://sci2s.ugr.es/imbalanced  
  - Computer Vision:  Models, Learning, and Inference (Simon J.D. Prince):  
    http://www.computervisionmodels.com/  
  - Oversampling with VAEs:  
    https://towardsdatascience.com/oversampling-with-vaes-e410887fe51  
